In [ ]:
from collections import defaultdict
from datetime import datetime
import os

import dotenv
import pandas as pd


In [ ]:
# !pip install -e ../../../pipeline
# !pip install -e ../../../../data-inclusion-schema

from data_inclusion.scripts.tasks import grist
from data_inclusion.schema.models import Service


In [ ]:
dotenv.load_dotenv(dotenv.find_dotenv())

GRIST_API_TOKEN = os.environ["GRIST_API_TOKEN"]
GRIST_API_URL = "https://grist.incubateur.anct.gouv.fr/api"
WORKSPACE_ID = "1640"
DOCUMENT_NAME = "monenfant"


In [ ]:
grist_client = grist.GristClient(base_url=GRIST_API_URL, token=GRIST_API_TOKEN)

document_id = grist_client.create_document(
    workspace_id=WORKSPACE_ID, document_name=DOCUMENT_NAME
)


In [ ]:
grist_client.create_table(
    document_id=document_id,
    table_name="Typologie_de_structures",
    columns=[
        {"id": "value", "fields": {"label": "valeur", "type": "Text"}},
        {"id": "label", "fields": {"label": "label", "type": "Text"}},
    ],
)


In [ ]:
typologies_df = pd.read_csv(
    "../../../pipeline/dbt/seeds/schema/typologies_de_structures.csv",
    dtype=str,
)


In [ ]:
# attention: pas idempotent

grist_client.add_records(
    document_id=document_id,
    table_id="Typologie_de_structures",
    records=[
        {"fields": typologie_dict}
        for typologie_dict in typologies_df[["value", "label"]].to_dict(
            orient="records"
        )
    ],
)


In [ ]:
grist_client.create_table(
    document_id=document_id,
    table_name="Structures",
    columns=[
        {
            "id": value.name,
            "fields": {
                "label": value.name,
                "type": defaultdict(
                    lambda: "Text",
                    **{
                        "longitude": "Numeric",
                        "latitude": "Numeric",
                        "date_maj": "DateTime:Europe/Paris",
                        "typologie": "Ref:Typologie_de_structures",
                        "antenne": "Bool",
                    },
                )[value.type_.__name__],
            },
        }
        for _, value in Service.__fields__.items()
    ],
)


In [ ]:
grist_client.create_table(
    document_id=document_id,
    table_name="Services",
    columns=[
        {
            "id": value.name,
            "fields": {
                "label": value.name,
                "type": defaultdict(
                    lambda: "Text",
                    **{
                        "cumulable": "Bool",
                        "contact_public": "Bool",
                        "longitude": "Numeric",
                        "latitude": "Numeric",
                        "date_creation": "DateTime:Europe/Paris",
                        "date_suspension": "DateTime:Europe/Paris",
                        "date_maj": "DateTime:Europe/Paris",
                    },
                )[value.type_.__name__],
            },
        }
        for _, value in Service.__fields__.items()
    ],
)
